In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import json
import plotly.express as px
import geopandas as gpd

init_notebook_mode(connected=True)  

# Import Data

## Encounter Data

In [ ]:
cond = pd.read_csv('Encounter_Conditions.csv')
#enc = pd.read_csv('EncounterData.csv', low_memory = False)
enc_condition = pd.read_csv('Enc_Condition.csv')

encc = pd.read_csv('Encounter_Cond_Data.csv')

ctrends=pd.read_csv('EncounterCondition.csv')

pop_reached = pd.read_csv('Population_Reached.csv')

## Referral Data

In [ ]:
re = pd.read_csv('Referral_Data.csv')

## Hear About Data

In [ ]:
hear = pd.read_csv('FinalHearAbout.csv')

## United States Census Data

In [ ]:
us_df = pd.read_csv('us_trends.csv')

## Tennessee Census Data 

In [ ]:
tn_df = pd.read_csv('tn_trends.csv')

## County Census Data

In [ ]:
gen_census = pd.read_csv('01general_census.csv')
age_census = pd.read_csv('01age_census.csv')
emp_census = pd.read_csv('01employment_census.csv')
dis_census = pd.read_csv('01disability_census.csv')
pov_census = pd.read_csv('01poverty_census.csv')

## Zip Census Data

In [ ]:
# need zip code data still (in the map data at the moment)

## Department of Health Data

In [ ]:
# still waiting to import

## County Shape File

In [ ]:
# need to import

## Zip Shape File

In [ ]:
# Import County Shape Files for Census Data
zip_shape = gpd.read_file('filtered_zip_shapefile.shp')
zip_shape.set_index('zip')
zip_shape = zip_shape.to_crs(epsg = 4326)

# Graph Functions

## Background

#### State Heat Map by County

In [ ]:
def county_map():
    #formatting the final shape file
    tn_shape = gpd.read_file('filtered_tn_shapefile.shp')
    tn_shape["geometry"] = (tn_shape.to_crs(tn_shape.estimate_utm_crs()).simplify(100).to_crs(tn_shape.crs))
    tn_shape.set_index('county')
    tn_shape = tn_shape.to_crs(epsg = 4326)
    geojson = tn_shape.__geo_interface__

    #plotting the choropleth
    fig = px.choropleth_mapbox(tn_shape,
                                geojson = tn_shape.geometry,
                                locations = tn_shape.index,
                                color = tn_shape.percent_un,
                                color_continuous_scale='RdYlBu_r',
                                center={"lat": 35.5, "lon": -86},
                                mapbox_style="white-bg", 
                                zoom=5.9,
                                #width = 1000,
                                #height = 600,
                                custom_data=[tn_shape['county'], tn_shape['population'],tn_shape['percent_un'],tn_shape['uninsured']])

    #editing the hovertemplate
    fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>County Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}%<br>Uninsured Population: %{customdata[3]}')
    #edit the title
    fig.update_layout(title='Percent Uninsured by County in Tennessee')
    #edit the bar label
    
    tullahoma_lon = -86.215324  # Longitude of Tullahoma, TN
    tullahoma_lat = 35.367072   # Latitude of Tullahoma, TN
    
    fig.add_trace(
        go.Scattermapbox(
            lat=[tullahoma_lat],
            lon=[tullahoma_lon],
            mode='markers',
            marker=dict(
                size=10,
                color='black'
            ),
            hovertemplate='<b> Partners for Healing</b><br>Main Clinic',
            showlegend=False
        )
    )
    
    fig.add_trace(
        go.Scattermapbox(
            lat=[35.47115882202067],
            lon=[-86.08416561438848],
            mode='markers',
            marker=dict(
                size=10,
                color='black'
            ),
            hovertemplate='<b> Partners for Healing</b><br>Secondary Clinic',
            showlegend=False
        )
    )
    
    fig.update_layout(coloraxis_colorbar_title='Percent Uninsured')

    return fig

#### US vs TN Trend Lines

In [ ]:
def tn_us_trend():
    import plotly.graph_objects as go

    #Create the line traces
    trace_tn = go.Scatter(x=tn_df['year'], y=tn_df['percent_uninsured'], name='TN',customdata=tn_df['population'])
    trace_us = go.Scatter(x=us_df['year'], y=us_df['percent_uninsured'], name='US',customdata=us_df['population'])

    # Customize the hovertemplate for each trace
    trace_tn.hovertemplate = '<b>Tennessee</b>'+'<br>Year: %{x}<br>Total Population: %{customdata}<br>Percent Uninsured: %{y}'
    trace_us.hovertemplate = '<b>United States</b>'+'<br>Year: %{x}<br>Total Population: %{customdata}<br>Percent Uninsured: %{y}'

    #Create the layout
    layout = go.Layout(
        title='Percent Uninsured Over Time',
        xaxis=dict(title='Year', range=['2012', '2021']), 
        yaxis=dict(title='Percent Uninsured')
    )

    #Combine the traces and layout into a figure
    fig = go.Figure(data=[trace_tn, trace_us], layout=layout)

    #Display
    return fig


## Clinic Usage

#### Clinic Usage by Month

In [ ]:
def monthly_usages(var):
    #Create bar plots for each year
    fig = go.Figure()

    years = monthly_visits['enc year'].unique()
    for year in years:
        df_year = monthly_visits[monthly_visits['enc year'] == var]
        visible = False if year != years[-1] else True
        fig.add_trace(go.Bar(x=df_year['enc month'], y=df_year['patientid'], name=str(year),
                            hovertemplate='Year: %{customdata[0]}<br>' +
                                           'Month: %{x}<br>' +
                                           'Patient Count: %{y}<extra></extra>',
                                            customdata=df_year,
                                            visible=visible))

    #Change layout
    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=list(range(1, 13)),  # Use a fixed range of values from 1 to 12 for all months
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        ),
        yaxis=dict(
            title='Number of Patients'
        ),
        title='Number of Patients per Month'
    )

    #Plot
    return fig


#### Clinic Usage Trend Line

In [ ]:
def patient_count_trends():
    #Create line graph
    fig = go.Figure()

    year_trends['Year'] = year_trends.index.year
    year_trends['Month'] = year_trends.index.strftime('%B')

    fig.add_trace(go.Scatter(
        x=year_trends.index,
        y=year_trends['patientid'],
        mode='lines',
        name='Patient Count',
        hovertemplate='Year: %{customdata[0]}<br>' +
                      'Month: %{customdata[1]}<br>' +
                      'Patient Count: %{y}<extra></extra>',
        customdata=year_trends[['Year', 'Month']]
    ))
    #Update layout
    fig.update_layout(
        xaxis=dict(
            title='Year',
        ),
        yaxis=dict(
            title='Percent Uninsured'
        ),
        title='Tennessee Percent Uninsured vs United States Percent Uninsured'
    )
    #Plot
    return fig


#### Bar Population Reached

In [ ]:
def reached_bar():
    # Group the data by 'county' and 'year' and calculate the mean of 'percent_reached'
    grouped_df = pop_reached.groupby(['county', 'year'])['percent_reached'].mean().reset_index()

    # Create a bar graph with grouped bars
    fig = go.Figure()

    for county in grouped_df['county'].unique():
        county_data = grouped_df[grouped_df['county'] == county]
        fig.add_trace(go.Bar(
            x=county_data['year'],
            y=county_data['percent_reached'],
            name=county
        ))

    # Set the title and axis labels
    fig.update_layout(
        title='Percent Reached by County',
        xaxis_title='Year',
        yaxis_title='Percent Reached'
    )

    # Show the figure
    return fig


#### Pie Population Reached

In [ ]:
import plotly.graph_objects as go

def reached_pie(county, year):
    # Filter the dataframe for the specified county and year
    filtered_df = pop_reached[(pop_reached['county'] == county) & (pop_reached['year'] == year)]
    
    # Calculate the 'percent_not_reached'
    percent_reached = filtered_df['percent_reached'].values[0]
    percent_not_reached = 100 - percent_reached

    # Create the pie chart using Plotly
    fig = go.Figure(data=[go.Pie(labels=['Percent Reached', 'Percent Not Reached'],
                                 values=[percent_reached, percent_not_reached])])

    # Set the title
    fig.update_layout(title=f'Percentage Reached vs Not Reached - {county} ({year})')

    # Show the figure
    return fig


#### Referral Count Plot

In [ ]:
def referral_count(year):
    # Filter the DataFrame for the specific year
    re_specific_year = re[re['enc year'] == year]

    # Sort the 'order type grp' column
    sorted_re = re_specific_year['order type grp'].value_counts().sort_values(ascending=False)

    # Create
    fig = px.histogram(re_specific_year, y='order type grp', color_discrete_sequence=['blue'],
                       category_orders={'order type grp': sorted_re.index})

    # Hovertemplate
    fig.update_traces(hovertemplate='Referral Type: %{y}<br>Reference Count: %{x}<extra></extra>')

    # Change x-axis 
    x_range = [0, 7500]
    fig.update_xaxes(range=x_range)

    #Labels
    fig.update_layout(
        yaxis_title='Type of Referral',
        xaxis_title='Referral Count',
        title=f"Partners for Healing Referral Count for Year {year}"
    )

    # Plot
    return fig

#### ICD10 Counts

In [ ]:
# function that calculates the top 10 diagnoses at the clinic for a given year for a specific county
def top10icd10DiagPerCounty(year, county):
    # filter the data to be only for the specified year
    enc_year = encc[encc['year'] == year]
    enc_year = enc_year[enc_year['ptnt cnty f rsdnc'].isin(county)]
    # group the data such that each row is one patient and has a list of all diagnoses the patient has ever had
    patients_year = enc_year.groupby('patientid')['icd10'].unique().reset_index(name='all_unique_diag')
    # function that creates a dictionary between diagnoses and the amount of patients that have that diagnosis
    def count_diag(x):
        d={}
        for i in x:
            if i in d.keys():
                d[i]+=1
            else:
                d[i]=1
        return d
    # sorting the dictionary from most to least frequent diagnosis
    def value_getter(item):
        return item[1]
    lis = [a for b in patients_year['all_unique_diag'].tolist() for a in b]
    diag_top10 = count_diag(lis)
    popular_diag = sorted(diag_top10.items(), key = value_getter)[::-1]
    # defining a list of the top 10 diagnoses
    top_10_diag = popular_diag[:10]
    
    # making it a list of the diagnosis names
    top_10_diag = [x[0] for x in top_10_diag]
    
    # removing the codes from the names (or keeping just the nums depending on what you want to do)
    top_10_diag = [x.split(': ')[0] for x in top_10_diag]
    return top_10_diag


#### Referance Count Plot

In [ ]:
def hear_about():
    #sort the ha ref source counts
    sorted_ha = hear['ref source'].value_counts().sort_values(ascending=False)

    #create the figure
    fig = px.histogram(hear, y='ref source', color='ref source', color_discrete_sequence=['blue'], category_orders={'ref source': sorted_ha.index})

    #edit the hovertemplate
    fig.update_traces(hovertemplate='Reference Source: %{y}<br>Reference Count: %{x}<extra></extra>')


    #edit labels
    fig.update_layout(
        yaxis_title='Reference Source',
        xaxis_title='Reference Count',
        title='How did you hear about Partners for Healing?'
    )

    #plot
    return fig

## Demographics

#### Federal Poverty Level Distribution

In [ ]:
def distribution(year):
    #Create histograms
    data = ctrends[ctrends['enc year']==year]
    
    fig = px.histogram(data, x='ptnt  fpl', nbins=10, color='enc year', histnorm='percent')

    #Change layout
    fig.update_layout(
        xaxis=dict(title='Federal Poverty Level'),
        yaxis=dict(title='Percentage'),
        title='Distribution of Federal Poverty Level'
    )

    # Update bar labels
    fig.update_traces(xbins=dict(start=min(bin_ranges), end=250, size=50), overwrite=True)

    # Custom hover template
    fig.update_traces(hovertemplate='Federal Poverty Level: %{x}<br>' +
                                    'Percent: %{y}%<br>',
                      customdata=fig.data[0]['customdata'])

    #Hide legend
    for trace in fig.data:
        trace.update(legendgroup='', showlegend=False)

    #Only show 2023 distribution
    for i in range(0, len(fig.data)-1):
        fig.data[i].visible = False

    #Plot
    return fig

#### Gender Distribution

In [ ]:
def gender_dist(year):
    # getting data for that year
    cond_year = cond[cond['enc year'] == year]
    
    # grouping by patientid so there is one row per patient
    patients_year = cond_year.groupby('patientid').agg({
        'patientsex': 'first',
    }).reset_index()
    
    # get the gender counts
    count = patients_year['patientsex'].value_counts()
    
    # create the plot
    fig = px.pie(count, values = count.values, names = count.index, title = 'Gender Distribution')
    
    #hovertemplate
    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>Count: %{value}')
    
    return fig


#### Age Distribution

In [ ]:
def age_dist(year):
    # getting data for that year
    cond_year = cond[cond['enc year'] == year]
    # grouping by patientid so there is one row per patient
    patients_year = cond_year.groupby('patientid').agg({
        'patientdob': 'first',
    }).reset_index()
    # calculating the age of each patient and putting it in a list
    age_years = []
    for dob in patients_year['patientdob']:
        age_years.append(int(year) - int(dob.split('/')[-1]))
    # creating the new age col with each age
    patients_year['age'] = [int(x) for x in age_years]
    # get the age counts using value_counts
    age_counts = patients_year['age'].value_counts()
    # create the plot
    fig = px.bar(age_counts, x = age_counts.index, y = age_counts.values,
                 labels={
                     'index': "Age",
                     'y': "Number of Patients"
                 },
                 title = f"Age Distribution for the Year {year}")
    
        #hovertemplate
    fig.update_traces(
        hovertemplate='Age: %{x}<br>Count: %{y}')
    
    return fig

#### County Distribution

In [ ]:
def county_dist(year):
    cond_year = cond[cond['enc year'] == year]
    patients_county = cond_year.groupby('patientid').agg({
        'ptnt cnty f rsdnc': 'first',
    }).reset_index()
    counts = patients_county['ptnt cnty f rsdnc'].value_counts()
    fig = px.bar(counts, x = counts.index, y = counts.values,
                labels={
                    'x': 'County',
                    'y': 'Patient Counts',
                },
                title = "County Distribution")
    
        #hovertemplate
    fig.update_traces(
        hovertemplate='County: %{x}<br>Count: %{y}')
    
    return fig


## Condition

#### Condition of Patients by Age Range

In [ ]:
# function that calculates the top 10 diagnoses at the clinic for a given year for a specific county
def top10DiagPerCounty(year, county):
    # filter the data to be only for the specified year
    enc_year = enc_condition[enc_condition['year'] == year]
    enc_year = enc_year[enc_year['ptnt cnty f rsdnc'].isin(county)]
    # group the data such that each row is one patient and has a list of all diagnoses the patient has ever had
    patients_year = enc_year.groupby('patientid')['patientsnomedproblemcodedesc'].unique().reset_index(name='all_unique_diag')
    # function that creates a dictionary between diagnoses and the amount of patients that have that diagnosis
    def count_diag(x):
        d={}
        for i in x:
            if i in d.keys():
                d[i]+=1
            else:
                d[i]=1
        return d
    # sorting the dictionary from most to least frequent diagnosis
    def value_getter(item):
        return item[1]
    lis = [a for b in patients_year['all_unique_diag'].tolist() for a in b]
    diag_top10 = count_diag(lis)
    popular_diag = sorted(diag_top10.items(), key = value_getter)[::-1]
    
        # defining a list of the top 10 diagnoses
    top_10_diag = popular_diag[:10]
    
    # making it a list of the diagnosis names
    top_10_diag = [x[0] for x in top_10_diag]
    
    # removing the snomed codes from the names
    top_10_diag = [x.split(': ')[-1] for x in top_10_diag]
    return top_10_diag

In [ ]:
age_ranges = ['18-32', '33-48', '49-63']
# function that plots condition by age range graph by county
def condByAgeRangeAndCounty(year, county, var):
    
    # create a copy of the dataset
    enc_condition1 = enc_condition.copy()
    
    # choosing data from the specific year
    enc_condition1 = enc_condition1[enc_condition1['year'] == year]
    enc_condition1 = enc_condition1[enc_condition1['ptnt cnty f rsdnc'].isin(county)]
    
    # setting age col to be integers
    enc_condition1['age'] = enc_condition1['age'].astype(int)
    
    # creating an age range col
    enc_condition1['age range'] = enc_condition1['age'].apply(lambda x: age_ranges[0] if x >= 18 and x <= 32 else (age_ranges[1] if x >= 33 and x <= 48 else (age_ranges[2] if x >= 49 and x <= 63 else 'other')))
    
    if var == 'age':
        var ='age range'
        
    # specifying necessary columns and dropping null values
    df = enc_condition1[['patientsnomedproblemcodedesc', var, 'patientid', 'year']]
    df.dropna(inplace = True)
    
    # setting rows to be unique patients
    df1 = df.groupby(['patientsnomedproblemcodedesc', var, 'year'])['patientid'].nunique().reset_index()
    
    # removing the snomed codes from the diagnosis column
    df1['patientsnomedproblemcodedesc'] = df1['patientsnomedproblemcodedesc'].apply(lambda diag: diag.split(': ')[1])
    # only taking the rows in the top 10
    df1 = df1[df1['patientsnomedproblemcodedesc'].isin(top10DiagPerCounty(year, county))]
    # plot the graph
    cond_by_age_range = px.bar(df1, x = 'patientsnomedproblemcodedesc', y = 'patientid', color = var, barmode = 'group')
    
    cond_by_age_range.update_traces(
        hovertemplate='SNOMED-CT: %{x}<br>Count: %{y}')
    
    cond_by_age_range.update_layout(
        xaxis=dict(title='SNOMED_CT Condition'),
        yaxis=dict(title='Condition Count'),
        title=f"SNOMED-CT Condition Counts by {var}")
    
    cond_by_age_range.update_layout(xaxis={'categoryorder':'total descending'})
    
    return cond_by_age_range

#### Conditions by ICD10

In [ ]:
# function that calculates the top 10 diagnoses at the clinic for a given year for a specific county
def top10icd10DiagPerCounty(year, county):
    # filter the data to be only for the specified year
    enc_year = encc[encc['year'] == year]
    enc_year = enc_year[enc_year['ptnt cnty f rsdnc'].isin(county)]
    # group the data such that each row is one patient and has a list of all diagnoses the patient has ever had
    patients_year = enc_year.groupby('patientid')['icd10'].unique().reset_index(name='all_unique_diag')
    # function that creates a dictionary between diagnoses and the amount of patients that have that diagnosis
    def count_diag(x):
        d={}
        for i in x:
            if i in d.keys():
                d[i]+=1
            else:
                d[i]=1
        return d
    # sorting the dictionary from most to least frequent diagnosis
    def value_getter(item):
        return item[1]
    lis = [a for b in patients_year['all_unique_diag'].tolist() for a in b]
    diag_top10 = count_diag(lis)
    popular_diag = sorted(diag_top10.items(), key = value_getter)[::-1]
    # defining a list of the top 10 diagnoses
    top_10_diag = popular_diag[:10]
    
    # making it a list of the diagnosis names
    top_10_diag = [x[0] for x in top_10_diag]
    
    # removing the codes from the names (or keeping just the nums depending on what you want to do)
    top_10_diag = [x.split(': ')[0] for x in top_10_diag]
    return top_10_diag

In [ ]:
# function that plots condition by age range graph by county
def icd10ByAgeRangeAndCounty(year, county, variable):
    
    # create a copy of the dataset
    enc_condition1 = encc.copy()
    
    # choosing data from the specific year
    enc_condition1 = enc_condition1[enc_condition1['year'] == year]
    enc_condition1 = enc_condition1[enc_condition1['ptnt cnty f rsdnc'].isin(county)]
    
    # setting age col to be integers
    enc_condition1['age'] = enc_condition1['age'].astype(int)
    
    # creating an age range col
    enc_condition1['age range'] = enc_condition1['age'].apply(lambda x: age_ranges[0] if x >= 18 and x <= 32 else (age_ranges[1] if x >= 33 and x <= 48 else (age_ranges[2] if x >= 49 and x <= 63 else 'other')))
    
        
    if variable == 'age':
        variable='age range'
        
    
    # specifying necessary columns and dropping null values
    df = enc_condition1[['icd10', variable, 'patientid', 'year']]
    df.dropna(inplace = True)
    
    # setting rows to be unique patients
    df1 = df.groupby(['icd10', variable, 'year'])['patientid'].nunique().reset_index()
    
    # removing the codes from the diagnosis column (or having just the codes depending on what you want to do)
    df1['icd10'] = df1['icd10'].apply(lambda diag: diag.split(': ')[0])
    # only taking the rows in the top 10
    df1 = df1[df1['icd10'].isin(top10icd10DiagPerCounty(year, county))]
    
    # plot the graph
    cond_by_age_range = px.bar(df1, x = 'icd10', y = 'patientid', color = variable, barmode = 'group')
    
    cond_by_age_range.update_traces(
        hovertemplate='ICD10 Code: %{x}<br>Count: %{y}')
    
    cond_by_age_range.update_layout(
        xaxis=dict(title='ICD10 Condition'),
        yaxis=dict(title='Condition Count'),
        title=f"ICD10 Condition Counts by {variable}")
    
    cond_by_age_range.update_layout(xaxis={'categoryorder':'total descending'})
    return cond_by_age_range

## Eligibility

#### Zip Code Heat Map for Census Data

In [ ]:
def zip_map(var):   
    #formatting the final shape file
    geojson = zip_shape.__geo_interface__

    #plotting the choropleth
    fig = px.choropleth_mapbox(zip_shape,
                                geojson = zip_shape.geometry,
                                locations = zip_shape.index,
                                color = zip_shape[var],
                                color_continuous_scale='RdYlBu_r',
                                center={"lat": 35.5, "lon": -86.01},
                                mapbox_style="white-bg", 
                                zoom=8,
                                width = 800,
                                height = 650,
                                custom_data=[zip_shape['county_nam'],zip_shape['city'], zip_shape['zip'],zip_shape['population'],zip_shape['percent_un'],zip_shape['uninsured']]
                            )
    if var == 'uninsured':
        name = 'Uninsured Population by Zip Code'
        ylabel = 'Uninsured Population'
        
    else:
        name = 'Percent Uninsured by Zip Code'
        ylabel = "Percent Uninsured"
    
    #editing the hovertemplate
    fig.update_traces(hovertemplate='<b>%{customdata[0]}'+' County'+'</b><br>City: %{customdata[1]}<br>Zip Code: %{customdata[2]}<br>Population: %{customdata[3]}<br>Percent Uninsured: %{customdata[4]}<br>Uninsured Population: %{customdata[5]}')
    #edit the title
    fig.update_layout(title=name)
    #edit the bar label
    fig.update_layout(coloraxis_colorbar_title=ylabel)
    
    tullahoma_lon = -86.215324  # Longitude of Tullahoma, TN
    tullahoma_lat = 35.367072   # Latitude of Tullahoma, TN
    
    fig.add_trace(
        go.Scattermapbox(
            lat=[tullahoma_lat],
            lon=[tullahoma_lon],
            mode='markers',
            marker=dict(
                size=10,
                color='black'
            ),
            hovertemplate='<b> Partners for Healing</b><br>Main Clinic',
            showlegend=False
        )
    )
    
    fig.add_trace(
        go.Scattermapbox(
            lat=[35.47115882202067],
            lon=[-86.08416561438848],
            mode='markers',
            marker=dict(
                size=10,
                color='black'
            ),
            hovertemplate='<b> Partners for Healing</b><br>Secondary Clinic',
            showlegend=False
        )
    )
    
    #plot
    return fig

#### County Uninsured Trend Lines

In [ ]:
def create_line_graph(var):
    filtered_data = gen_census[gen_census['county'].isin(counties_to_show)]
   
    if var == 'uninsured':
        name = 'Uninsured Population by County'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured by County'
        ylabel = "Percent Uninsured"
        
    fig = px.line(
        filtered_data,
        x='year',
        y=var,
        color='county',
        title=name,
        custom_data=['county', 'population', 'percent_uninsured', 'uninsured'],
        color_discrete_sequence=px.colors.qualitative.Plotly
    )

    
    fig.update_layout(
        xaxis_title='Year',  # Set the x-axis title
        yaxis_title=ylabel,  # Set the y-axis title
        title_font=dict(size=20),  # Adjust the title font size
        legend_title='County',  # Set the legend title
        legend_title_font=dict(size=14),  # Adjust the legend title font size
        legend_font=dict(size=12),  # Adjust the legend font size
    )
    
    fig.update_traces(
        mode='lines+markers',
        hovertemplate="<b>%{customdata[0]}</b><br>Year: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Uninsured Population: %{customdata[3]}",
    )
    
    #Set x axis format
    fig.update_layout(xaxis=dict(range=[2018, 2021], tickmode='linear', dtick=1))
    
    return fig




#### Uninsured Rates by Age

In [ ]:
def generate_age_graph(year, var):
    filtered_data = age_census[age_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['age'].isin(['6-18', '19-25', '26-34', '35-44', '45-54', '55-64'])]
    filtered_data = filtered_data[filtered_data['year'] == year]
    years = filtered_data['year'].unique()

    #Create figure
    fig = go.Figure()

    # Create bar plots for each year and county
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True  # Set visibility to False for all bars except the last year
            fig.add_trace(go.Bar(x=county_data['age'],
                                y=county_data[var], name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Age Range: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Population Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))
    if var == 'uninsured':
        name = 'Uninsured Population per County by Age'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Age'
        ylabel = "Percent Uninsured"
    #Update layout with x-axis labels
    fig.update_layout(
       xaxis={'title': 'Age Range'},
        yaxis={'title': ylabel},
        title=name
    )
    
    return fig

#### Uninsured Rates by Employment Status

In [ ]:
def generate_emp_graph(year,var):
    filtered_data = emp_census[emp_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['employment_status (19-64)'].isin(['employed','unemployed', 'not in labor force'])]
    filtered_data = filtered_data[filtered_data['year']==year]

    years = filtered_data['year'].unique()

    #Create empty figure
    fig = go.Figure()

    #Create bar plots for each year
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True
            fig.add_trace(go.Bar(x=county_data['employment_status (19-64)'], 
                                y=county_data[var],
                                name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Employment Status: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Populaton Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))

    if var == 'uninsured':
        name = 'Uninsured Population per County by Employment Status'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Employent Status'
        ylabel = "Percent Uninsured"
        
    #Update layout with x-axis
    fig.update_layout(
        xaxis={'title': 'Employment Status'},
        yaxis={'title': ylabel},
        title=name
    )
    return fig

#### Uninsured Rates by Disibility Status

In [ ]:
def generate_dis_graph(year,var):
    filtered_data = dis_census[dis_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['disability_status'].isin(['disability','no disability'])]
    filtered_data = filtered_data[filtered_data['year']==year]
    years = filtered_data['year'].unique()

    #Create empty figure
    fig = go.Figure()

    #Create bar plots for each year
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True
            fig.add_trace(go.Bar(x=county_data['disability_status'], 
                                y=county_data[var], 
                                name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Disability Status: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Populaton Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))

    if var == 'uninsured':
        name = 'Uninsured Population per County by Disability Status'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Disability Status'
        ylabel = "Percent Uninsured"
    
    #Update layout with x-axis labels
    fig.update_layout(
        xaxis={'title': 'Disability Status'},
        yaxis={'title': ylabel},
        title=name
    )
    return fig

#### Uninsured Rates by Federal Poverty Level

In [ ]:
def generate_pov_graph(year,var):
    filtered_data = pov_census[pov_census['county'].isin(counties_to_show)]
    filtered_data = filtered_data[filtered_data['poverty_level (%)'].isin(['<138','138-399','>400'])]
    filtered_data = filtered_data[filtered_data['year']==year]

    years = filtered_data['year'].unique()

    #Create empty figure
    fig = go.Figure()

    #Create bar plots for each year
    for year in years:
        year_data = filtered_data[filtered_data['year'] == year]
        for county in counties_to_show:
            county_data = year_data[year_data['county'] == county]
            visible = False if year != years[-1] else True
            fig.add_trace(go.Bar(x=county_data['poverty_level (%)'],
                                y=county_data[var],
                                name=county,
                                hovertemplate="<b>%{customdata[0]}</b><br>Year: " + str(year) + "<extra></extra><br>Poverty Level: %{x}<br>Total Population: %{customdata[1]}<br>Percent Uninsured: %{customdata[2]}<br>Populaton Uninsured: %{customdata[3]}",
                                customdata=county_data[['county','population','percent_uninsured','uninsured']],
                                visible=visible))
    if var == 'uninsured':
        name = 'Uninsured Population per County by Federal Poverty Level (Percentage)'
        ylabel = 'Uninsured Population Count'
        
    else:
        name = 'Percent Uninsured per County by Federal Poverty Level (Percentage)'
        ylabel = "Percent Uninsured"
    
    #Update layout with x-axis labels
    fig.update_layout(
        xaxis={'title': 'Poverty Level Percentage'},
        yaxis={'title': ylabel},
        title=name
    )
    return fig

## ER Usage

In [ ]:
def top10diag():
    top_10_diagnoses = df['Diag1'].value_counts().nlargest(10)
    # Dictionary mapping ICD codes to diagnosis names
    icd_to_diagnosis = {
        "N390": "UTI",
        "R109": "Abdominal pain",
        "R0789": "Other chest pain",
        "R079": "Chest pain",
        "M545": "Low back pain",
        "J069": "Acute URI",
        "R112": "Nausea with vomiting",
        "R51": "Headache",
        "J209": "Acute bronchitis",
        "A419": "Sepsis"
    }
    # Mapping diagnosis names to the top 10 diagnoses
    top_10_diagnoses_names = top_10_diagnoses.index.map(icd_to_diagnosis)
    # Creating the bar graph using Plotly
    fig = px.bar(x=top_10_diagnoses_names, y=top_10_diagnoses.values, labels={'x': 'Diagnosis', 'y': 'Count'})
    fig.update_layout(title="Top 10 Diagnoses",
                      xaxis_title="Diagnosis",
                      yaxis_title="Count")
    return fig

In [ ]:
def preventable_diag_zipcode():
    # Specify the preventable diagnoses
    preventable_diagnoses = ['I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19']
    # Filter the DataFrame based on the preventable diagnoses
    filtered_data = df[df['Diag1'].isin(preventable_diagnoses)]
    # Combine specified zip codes into one category 'Other'
    filtered_data.loc[filtered_data['Patient_Zip'].isin(['37376', '37382', '37349']), 'Patient_Zip'] = 'Other'
    # Create a histogram of Patient_Zip counts for the preventable diagnoses
    fig = px.histogram(filtered_data, x='Patient_Zip', title='Count of Preventable Diagnoses by Zip Code')
    # Display the interactive histogram
    fig.show()

In [ ]:
def preventable_diag_county():
    # Specify the zip codes and their respective counties
    zip_county_mapping = {
        '37318': 'Franklin County',
        '37398': 'Franklin County',
        '37330': 'Franklin County',
        '37306': 'Franklin County',
        '37345': 'Franklin County',
        '37324': 'Franklin County',
        '37375': 'Franklin County',
        '37376': 'Franklin County',
        '37355': 'Coffee County',
        '37388': 'Coffee County',
        '37342': 'Coffee County',
        '37349': 'Coffee County',
        '37352': 'Moore County'
    }
    # Specify the preventable diagnoses
    preventable_diagnoses = [
        'I20', 'I240', 'I248', 'I249', 'J45', 'J13', 'J14', 'J153', 'J154', 'J157', 'J159', 'J16', 'J18', 'L03', 'L04', 'L08', 'L88', 'L980', 'J20', 'J40', 'J41', 'J42', 'J43', 'J44', 'J47', 'I50', 'I110', 'J810', 'R56', 'E86', 'E109', 'E119', 'E101', 'E131', 'E110', 'E130', 'E10641', 'E11641', 'E106', 'E116', 'E108', 'E118', 'K529', 'K5289', 'G40', 'I10', 'I119', 'E162', 'N10', 'N11', 'N12', 'N70', 'N73', 'A150', 'A155', 'A159', 'H66', 'J02', 'J03', 'J06', 'J312', 'A154', 'A156', 'A158', 'A17', 'A18', 'A19'
    ]
    # Filter the DataFrame based on zip codes and preventable diagnoses
    filtered_data = df[df['Patient_Zip'].isin(zip_county_mapping.keys()) & df['Diag1'].isin(preventable_diagnoses)]
    # Assign the respective counties to each zip code
    filtered_data['County'] = filtered_data['Patient_Zip'].map(zip_county_mapping)
    # Create an interactive histogram
    fig = px.histogram(filtered_data, x='County', title='Count of Preventable Diagnoses by County')
    # Display the interactive histogram
    fig.show()

## Rural vs Urban

# Dash

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

testing = ['Coffee', 'Franklin', 'Moore']
#---------------------------------------------------------------------------------------------------------------------
# ADDED LISTS 
#---------------------------------------------------------------------------------------------------------------------

#List to filter out the Tennessee counties
counties_to_show = ['Coffee County', 'Franklin County', 'Moore County', 'Bedford County', 'Cannon County', 'Warren County', 'Grundy County', 'Marion County']

#---------------------------------------------------------------------------------------------------------------------
# ENCOUNTER DATA CLEANING
#---------------------------------------------------------------------------------------------------------------------


## CREATING THE AGE COL (for enc_condition)
# description: a column that represents the patients age at the time of the clinic encounter
from datetime import datetime
# Format according to datetime module
dob = pd.to_datetime(enc_condition['patientdob'], format='%m/%d/%Y')
encdate = pd.to_datetime(enc_condition['cln enc date'], format='%m/%d/%Y')
# Calculate the age in days
age_days = (encdate - dob).dt.days
# Convert age from days to years
age_years = age_days // 365.25
# Create the new column and make the ages ints
enc_condition['age'] = [int(x) for x in age_years]
## CREATING THE YEAR COL (for enc_condition)
enc_condition['year'] = enc_condition['cln enc date'].apply(lambda x: x.split('/')[-1])
enc_condition['year']=enc_condition['year'].astype(int)

# function that calculates the top 10 diagnoses at the clinic for a given year


#---------------------------------------------------------------------------------------------------------------------
# ADDED LISTS 
#---------------------------------------------------------------------------------------------------------------------
#List to filter out the tennessee counties
counties_to_show = ['Coffee County', 'Franklin County', 'Moore County', 'Bedford County', 'Cannon County', 'Warren County', 'Grundy County', 'Marion County']

#graph age ranges combination
age_ranges = ['18-32', '33-48', '49-63']

# Create the Dash application
app = dash.Dash(__name__, suppress_callback_exceptions=True, external_stylesheets=[dbc.themes.BOOTSTRAP])

#---------------------------------------------------------------------------------------------------------------------
# SECTION FOR IMPORTED CSV AND SHAPE FILES
#---------------------------------------------------------------------------------------------------------------------



#Import Condition Encounter Data
ctrends=pd.read_csv('EncounterCondition.csv')
#Groupby 'enc year' and 'enc month' and count'patientid'
monthly_visits = ctrends.groupby(['enc year', 'enc month'])['patientid'].count().reset_index()
years = monthly_visits['enc year'].unique()
#Convert 'enc month' column to numeric
ctrends['enc month'] = pd.to_numeric(ctrends['enc month'])
#Combine 'enc year' and 'enc month'
ctrends['enc date'] = pd.to_datetime(ctrends['enc year'].astype(str) + '-' + ctrends['enc month'].astype(str), format='%Y-%m')
#Groupby 'enc date' and count 'patientid'
year_trends = ctrends.groupby('enc date')['patientid'].count().reset_index()
#Resample to monthly frequency
year_trends = year_trends.resample('M', on='enc date').sum()
#Define bin ranges
bin_ranges = list(range(0, 451, 50))

#---------------------------------------------------------------------------------------------------------------------
# SECTION FOR THE SIDE BAR
#---------------------------------------------------------------------------------------------------------------------
#app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
#Styling sidebar

SIDEBAR_STYLE= {
    "position": "fixed",
    "top":0,
    "left":0,
    "bottom":0,
    "width": "12rem",
    "padding":"2rem 1rem",
    "background-color": "#F8F9FA",
    "overflow": "scroll",
}
# location of tab information
CONTENT_STYLE = {
    "margin-left": "16rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",}
# Create the Sidebar
sidebar = html.Div(
    [
        html.H2("PfH", className="display-1"),
        html.Hr(),
        html.P ('Home Page'),
        dbc.Nav([
                dbc.NavLink("Background", href="/", active = "exact"),
        ],
            vertical = True,
            pills = True
        ),
        html.Hr(),
        html.P(
            'Partners for Healing', className='lead'
        ),
        dbc.Nav(
            [
                dbc.NavLink("Clinic Usage", href="/clinic_usage", active = "exact"),
                dbc.NavLink("Demographics", href="/demographics", active="exact"),
                dbc.NavLink("Conditions", href="/conditions", active="exact"),
            ],
            vertical = True,
            pills = True,
        ),
        html.Hr(),
        html.P('County', className='lead'),
        dbc.Nav(
            [
                dbc.NavLink("Eligibility", href="/population", active = "exact"),
            ],
            vertical = True,
            pills = True,
        ),
        html.Hr(),
        html.P('Hospital Insights', className='lead'),
        dbc.Nav(
            [
                dbc.NavLink("ER Usage", href="/ER", active = "exact"),
                dbc.NavLink("Rural vs Urban", href="/rural_urban", active="exact"),
                dbc.NavLink("Test 8", href="/test-8", active="exact"),
            ],
            vertical = True,
            pills = True,
        ),
    ],
    style= SIDEBAR_STYLE,
)

# Create the content container
content = html.Div(id="page-content", style=CONTENT_STYLE)

# Define the layout
app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content
])

#---------------------------------------------------------------------------------------------------------------------
# CLINIC USAGE CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------

@app.callback(
    Output("monthly-use", "figure"),
    Input("monthly-use-slider", "value")
)
def update_monthly_usages(var):
    fig = monthly_usages(var)
    return fig

@app.callback(
    Output("reached-pie", "figure"),
    [Input("reached-pie-slider", "value"),
    Input("reached-pie-dropdown", "value"),
    ]
)
def update_reached_pie(year, var):
    fig = reached_pie(var, year)
    return fig


@app.callback(
    Output("poverty-dis", "figure"),
    Input("poverty-dis-slider", "value")
)
def update_distribution(year):
    fig = distribution(year)
    return fig

@app.callback(
    Output("referral-count", "figure"),
    Input("referral-count-slider", "value")
)
def update_referral_count(year):
    fig = referral_count(year)
    return fig

#---------------------------------------------------------------------------------------------------------------------
# DEMOGRAPHICS CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output("gender-dist", "figure"),
    Input("gender-dist-slider", "value")
)
def update_gender_dist(var):
    fig = gender_dist(var)
    return fig

@app.callback(
    Output("age-dist", "figure"),
    Input("age-dist-slider", "value")
)
def update_age_dist(var):
    fig = age_dist(var)
    return fig

@app.callback(
    Output("county-dist", "figure"),
    Input("county-dist-slider", "value")
)
def update_county_dist(var):
    fig = county_dist(var)
    return fig

#---------------------------------------------------------------------------------------------------------------------
# CONDITIONS CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------

@app.callback(
    Output("con-snomed", "figure"),
    [Input("con-snomed-slider", "value"),
#     Input("con-snomed-county-dropdown", "value"),
     Input("con-snomed-dropdown", "value")
    ]
)
def update_condByAgeRangeAndCounty(year, var):
    fig = condByAgeRangeAndCounty(year, testing, var)
    return fig

@app.callback(
    Output("con-icd", "figure"),
    [Input("con-icd-slider", "value"),
#     Input("con-snomed-county-dropdown", "value"),
     Input("con-icd-dropdown", "value")
    ]
)
def update_icd10ByAgeRangeAndCounty(year, variable):
    fig = icd10ByAgeRangeAndCounty(year, testing, variable)
    return fig

#---------------------------------------------------------------------------------------------------------------------
# ELIGIBILITY CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------

@app.callback(
    Output("gen-line", "figure"),
    Input("line-dropdown", "value")
)
def update_line_graph(var):
    fig = create_line_graph(var)
    return fig

@app.callback(
    Output("zip-map", "figure"),
    Input("zip-dropdown", "value")
)
def update_zip_map(var):
    fig = zip_map(var)
    return fig

@app.callback(
    Output("age-graph", "figure"),
    [Input("year-slider", "value"),
    Input("age-dropdown", "value")]
)
def update_age_graph(year,var):
    fig = generate_age_graph(year,var)
    return fig

@app.callback(
    Output("emp-graph", "figure"),
    [Input("year-slider-emp", "value"),
    Input("emp-dropdown", "value")]
)
def update_emp_graph(year,var):
    fig = generate_emp_graph(year,var)
    return fig

@app.callback(
    Output("dis-graph", "figure"),
    [Input("year-slider-dis", "value"),
    Input("dis-dropdown", "value")]
)
def update_dis_graph(year,var):
    fig = generate_dis_graph(year,var)
    return fig

@app.callback(
    Output("pov-graph", "figure"),
    [Input("year-slider-pov", "value"),
    Input("pov-dropdown", "value")]
)
def update_pov_graph(year,var):
    fig = generate_pov_graph(year,var)
    return fig


#---------------------------------------------------------------------------------------------------------------------
# ER USE CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# RURAL VS URBAN CALLBACKS AND CALLBACK FUNCTIONS
#---------------------------------------------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------------------------------------
# MAIN CALLBACK AND CALLBACK FUNCTION
#---------------------------------------------------------------------------------------------------------------------
@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)
def render_page_content(pathname):
#---------------------------------------------------------------------------------------------------------------------
# BACKGROUND PATHNAME
#---------------------------------------------------------------------------------------------------------------------
    if pathname == "/":
        return [
            #Title
            html.H1('Bridging The Health Care Coverage Gap', style={'textAlign': 'center'}),
            #Description
            html.P("In Tennessee, ~1 million individuals (12% of the population) as of 2022, lack health insurance coverage, placing the state in the bottom 18% of uninsured rates nationwide. This issue is particularly prevalent in rural areas, as they are medically underserved and have limited access to primary care services. Rural patients are often substituting primary care with ER services for non-emergent and preventable conditions, straining the healthcare system. Lack of primary care is not only costly but it also worsens a patients quality of life. Our partner, Partners for Healing (PfH), a free clinic in Tullahoma, works to provide primary care to the working uninsured population of Franklin, Coffee, and Moore County. To support PfH in maximizing their impact and reaching more individuals, we  leveraged data science to target expansion opportunities and optimize patient outreach effectively."),
            #Line
            html.Hr(),
            #County Heat Map
            dcc.Graph(id='state-graph',
                      figure=county_map()),
            #Data Source
            html.P('(US Census Data-2021)'),
            #Description
            html.P('This map displays the uninsured percentage of Tennessee separated by counties. It is interactive and allows the user to see the name of the county along with its population statistics when the cursor is hovering over a specific county. The user is also able to zoom in and out, but also auto scale the map. There are a total of 39 counties that are above the state average of 10.5, with hotspots for high uninsured percentages in rural counties, specifically Van Buren, Bledsoe County, and Perry County.'),
            #Line
            html.Hr(),
            #Unisnured Trends
            dcc.Graph(id='trends-graph',
                      figure=tn_us_trend()),
            #Data Source
            html.P('(US Census Data)'),
            #Description
            html.P('This graph displays the uninsured percentage trend line comparison between the United States and Tennessee. Both of these lines show a downward trend from 2012 to 2020 which starts around 2012. This timeline corresponds to the time in which the Affordable Care Act went into effect in March of 2010. Tennessee also has a lower uninsured percentage until around 2015. This timeline corresponds to the expansion of Medicaid in which Tennessee did not participate in. It is also noticeable that Tennessee is starting to trend up between the years 2020 and 2021'),
            #Line
            html.Hr(),
            
            html.P("Information of PfH and their clinic as well as their mission statement"),
            html.P("For the public dashboard we will add information about datalab and quick run through of our team."),
            html.P("Will add hyperlinks to datalab website and add emails and etc")
        ]
#---------------------------------------------------------------------------------------------------------------------
# CLINIC USAGE PATHNAME
#---------------------------------------------------------------------------------------------------------------------
    elif pathname == "/clinic_usage":
        return [
            #Heading
            html.H1("PfH Clinic Usage Throughout Years", style={'textAlign': 'center'}),
            #Description
            html.P("Tab Description"),
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#Clinic Monthly Usage
            #Graph
            dcc.Graph(id='monthly-use'),
            #Slider
            dcc.Slider(
                id="monthly-use-slider",
                min=ctrends['enc year'].min(),
                max=ctrends['enc year'].max(),
                step=1,
                value=ctrends['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Data Source
            html.P('(Partners for Healing Data)'),
            #Description
            html.P("The bar graph above displays the monthly encounter counts for the Partners for Healing clinic from 2016 until 2023. It is noticeable that the clinic had higher patient counts between the end of 2019 and 2021 with 2021 having significantly higher patient encounters than any other year. There is not a noticeable month that tends to be higher than others throughout the years, but there are a few such as January, March, July, and October that tend to be the highest months in each year."),
            #Lines
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#Clinic Patient Count Trend
            #Graph
            dcc.Graph(id='bargraph',
                       figure=patient_count_trends()),
            #Data Source
            html.P('(Partners for Healing Data)'),
            #Description
            html.P("The trend line above displays the overall usage of the clinic since 2016. The clinic had an upward trend from 2016 until March of 2021. The clinic then had an extreme decline in patient encounters after this month. This date corresponds around the same time as the Covid-19 pandemic that started in 2020. The clinic's usage may have been affected by the severity of the pandemic."),
            #Line
            html.Hr(),
            
            
#---------------------------------------------------------------------------------------------------------------------
#Reached Population Bar         
            #Graph
            dcc.Graph(id='reached-bar',
                       figure=reached_bar()),
            #Data Source
            html.P('(Partners for Healing Data and US Census Data)'),
            #Description
            html.P("This plot displays the percentage of the uninsured population that the Partners for Healing clinic is reaching from 2018-2021. It is created using census data and PfH encounter data. It only displays the three counties that the clinic serves. All three of the counties do not have an increase or decrease in the percentage of the population that they have reached, but the clinic has been able to reach more people in Coffee County. This is likely because the clinic is located in Coffee County, but it is right on the border of Franklin County.") ,   
            #Line
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#Reached Population Pie           
            #Graph
            dcc.Graph(id='reached-pie'),
            #Slider
            dcc.Slider(
                id="reached-pie-slider",
                min=pop_reached['year'].min(),
                max=pop_reached['year'].max(),
                step=1,
                value=pop_reached['year'].max(),
                marks={year: str(year) for year in range(2018, 2022)},
                className="slider"
            ),
            #Space
            html.P(''),
            #Dropdown
            dcc.Dropdown(
                id='reached-pie-dropdown',
                options=[
                    {'label': 'Coffee County', 'value': 'Coffee County'},
                    {'label': 'Franklin County', 'value': 'Franklin County'},
                    {'label': 'Moore County', 'value': 'Moore County'},
                ],
                value='Coffee County'
            ),
            #Space
            html.P(''),
            #Data Source
            html.P('(Partners for Healing Data and US Census Data)'),
            #Description
            html.P("This pie chart allows the user to isolate the percentage of the uninsured population that the Partners for Healing clinic reaches by year and county. It is very noticeable that the clinic does not reach a large population in Franklin or Moore County. They do in fact reach a larger population in Coffee County where they are located, but there is no increase in the percentage that they reach."),
            #Line
            html.Hr(),
            
            
#---------------------------------------------------------------------------------------------------------------------
#Hear About Counts
            #Graph
            dcc.Graph(id='hear-about',
                       figure=hear_about()),
            #Data Source
            html.P('(Partners for Healing Data)'),
            #Description
            html.P("This count plot displays the number of ways patients heard about the clinic. It is clear that most patients from the clinic heard about the clinic from someone else (word of mouth). There is also another group of ways that include the social services, hospital, and other clinics. These three ways are very important for the clinic's future. It might be worth creating more relationships between people in these three groups to increase their client base. Also some other outreaches such as social media do not seem to be working."),
            #Line
            html.Hr(),
        ]
#---------------------------------------------------------------------------------------------------------------------
# DEMOGRAPHICS PATHNAME
#---------------------------------------------------------------------------------------------------------------------
    elif pathname == "/demographics":
        return [
            #Title
            html.H1("PfH Clinic Demographics", style={'textAlign': 'center'}),
            #Description
            html.P("Information and patterns of the clinic's demographics. Highlights patient population characteristics like the most common age, ethnicity, and sex. An informational tool to be used by the clinic for quick statistics and better understanding of their clinic's patient population."),
            #Line
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#Gender Distribution
            #Graph
            dcc.Graph(id='gender-dist'),  
            #Slider
            dcc.Slider(
                id="gender-dist-slider",
                min=cond['enc year'].min(),
                max=cond['enc year'].max(),
                step=1,
                value=cond['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Space
            html.P(''),
            #Data Source
            html.P('(Partners for Healing Data)'),
            #Description
            html.P("This pie chart displays the Partners for Healing clinic's distribution of sex from 2016 to 2023. It is very noticeable that the clinic serves more women than men, but the percentages are becoming more equal in recent years. This is great for the clinic because it gives them a more even distribution of patients that are male or female."),
            #Line
            html.Hr(),

#---------------------------------------------------------------------------------------------------------------------
#Age Distribution
            #Graph
            dcc.Graph(id='age-dist'),  
            #Slider
            dcc.Slider(
                id="age-dist-slider",
                min=cond['enc year'].min(),
                max=cond['enc year'].max(),
                step=1,
                value=cond['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Space
            html.P(''),
            #Data Source
            html.P('(Partners for Healing Data)'),
            #Decription
            html.P("This visualization displays the age distribution of the Partners for Healing clinic’s population. It is clear that the majority of the patients that go to the clinic are between 50 and 60 years of age. This shows that older people are more willing to seek care, possibly because their symptoms are more noticeable. The clinic could possibly extend efforts to reach the younger population to promote preventative health care."),
            #Line
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#County Distribution    
            #Graph
            dcc.Graph(id='county-dist'), 
            #Slider
            dcc.Slider(
                id="county-dist-slider",
                min=cond['enc year'].min(),
                max=cond['enc year'].max(),
                step=1,
                value=cond['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Space
            html.P(''),
            #Data Source
            html.P('(Partners for Healing Data)'),
            #Description
            html.P("This visualization displays the counties in which the clinic’s patients are coming from by year. The majority of the clinic’s patients come from Coffee County with Franklin County being a distant second. This corresponds to the percentage of the population that the clinic reaches. It might be helpful for the clinic to try and reach more of the population in Franklin County."),
            #Line
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#Poverty Distribution
            #Graph
            dcc.Graph(id='poverty-dis'),
            #Slider
            dcc.Slider(
                id="poverty-dis-slider",
                min=ctrends['enc year'].min(),
                max=ctrends['enc year'].max(),
                step=1,
                value=ctrends['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Data Source
            html.P('(Partners for Healing Data)'),
            #Description
            html.P("This graph illustrates the distribution of clinic patients based on their federal poverty levels. It provides an overview of the percentage of patients falling into different poverty level categories, allowing for an understanding of the socio-economic composition of the clinic's patient population. The distributions show that the clinic mainly served people within 100-150 percent of the federal poverty level. Although in recent years, the clinic’s distribution has dispersed into a wider distribution between 0-250 percent of the federal poverty level."),
            #Line
            html.Hr(),
            
        ]
#---------------------------------------------------------------------------------------------------------------------
# CONDITIONS PATHNAME
#---------------------------------------------------------------------------------------------------------------------
    elif pathname == "/conditions":
        return [
            #Header
            html.H1("Patient Conditions", style={'textAlign': 'center'}),
            #Description
            html.P("Analysis of the most common conditions that appear in the PfH Clinic’s patients. Many of the conditions are common issues seen in rural areas. A lot of the health information found through patients highlight non-health issues that may lie in a community. For example, higher density of diabetes may convey low access to produce and grocery stores."),
            #Line
            html.Hr(),
        
#---------------------------------------------------------------------------------------------------------------------
#Condition SNOMED
            #Graph
            dcc.Graph(id='con-snomed'),
            #Slider
            dcc.Slider(
                id="con-snomed-slider",
                min=enc_condition['year'].min(),
                max=enc_condition['year'].max(),
                step=1,
                value=enc_condition['year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Space
            html.P(''),
            #Dropdown
            dcc.Dropdown(
                id='con-snomed-dropdown',
                options=[
                    {'label': 'Sex', 'value': 'patientsex'},
                    {'label': 'Age', 'value': 'age'},
                    {'label': 'Race', 'value': 'race'},
                    {'label': 'Ethnicity', 'value': 'ethnicity'},
                    {'label': 'Language', 'value': 'patient lang'},
                ],
                value='patientsex'
            ),
            #Space
            html.P(''),
            #Description
            html.P('(Partners for Healing Data)'),
            html.P("This visualization displays the top 10 SNOMED-CT by demographics and year. It is clear that the most common conditions include high blood pressure, high cholesterol, and diabetes. These are the highest in all demographic categories and are very common in the United States and can be very preventable."),
            #Line
            html.Hr(),  
            
#---------------------------------------------------------------------------------------------------------------------
#Condition ICD10
            #Graph
            dcc.Graph(id='con-icd'),
            #Slider
            dcc.Slider(
                id="con-icd-slider",
                min=enc_condition['year'].min(),
                max=enc_condition['year'].max(),
                step=1,
                value=enc_condition['year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Space
            html.P(''),
            #Dropdown
            dcc.Dropdown(
                id='con-icd-dropdown',
                options=[
                    {'label': 'Sex', 'value': 'patientsex'},
                    {'label': 'Age', 'value': 'age'},
                    {'label': 'Race', 'value': 'race'},
                    {'label': 'Ethnicity', 'value': 'ethnicity'},
                    {'label': 'Language', 'value': 'patient lang'},
                ],
                value='patientsex'
            ),
            #Space
            html.P(''),
            #Description
            html.P('(Partners for Healing Data)'),
            html.P('discription'),
            #Line
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#Referall Counts
            #Graph
            dcc.Graph(id='referral-count'),  
            #Slider
            dcc.Slider(
                id="referral-count-slider",
                min=re['enc year'].min(),
                max=re['enc year'].max(),
                step=1,
                value=re['enc year'].max(),
                marks={year: str(year) for year in range(2016, 2024)},
                className="slider"
            ),
            #Space
            html.P(''),
            #Data Source 
            html.P('(Partners for Healing Data)'),
            #Description
            html.P('This count plot displays the number of referrals the Partners for Healing clinic made separated by years. It is noticeable that prescriptions are the major need for a referral every year. This is likely due to the fact that each patient can receive multiple prescriptions in one encounter. Labs also had significant high counts compared to many other referrals.'),
            #Space
            html.Hr(),
        ]
#---------------------------------------------------------------------------------------------------------------------
# ELIGIBILITY PATHNAME
#---------------------------------------------------------------------------------------------------------------------
    elif pathname == "/population":
        return [
            html.H1("Population of Eligible Patients in Local Counties", style={'textAlign': 'center'}),
            
            html.P("In order to qualify for Partners for Healing’s free healthcare patients must be uninsured, working, between the ages of 18-64, and living/working in Franklin, Moore, and Coffee County. In this tab the graphs will be looking for these requirements in several counties. This information will be used by the clinic to understand where the possible patient populations lie. Additional counties surrounding Franklin, Moore, and Coffee were added to see if there is a large demand for the clinic's services outside of the areas they treat. "),
            html.Hr(),
            html.P("Instructions/Tips: Double clicking on the maps will reset the view. All of the graphs in this tab have dropdown menus that allow you to change the y-variable of the graphs from 'uninsured population' to 'uninsured percentage'."),
            html.Hr(),
        
#---------------------------------------------------------------------------------------------------------------------            
#Zip Heat Map
            dcc.Link('Simple Maps US Zips Website', href='https://simplemaps.com/data/us-zips'),
            dcc.Graph(id='zip-map'),
            dcc.Dropdown(
                id='zip-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},#must edit values
                    {'label': 'Percent Uninsured', 'value': 'percent_un'},
                ],
                value='percent_un'
            ),
            html.P(''),
            html.P("(US Census Data)"),
            html.P('This map displays the uninsured percentage and uninsured population for zip codes in the area. Mcminnville (Warren) and Shelbyville (Bedford) have significantly higher uninsured populations, while Sherwood (Franklin) and Monteagle (Marion) have higher percentage of uninsured people.'),
            html.Hr(),
#---------------------------------------------------------------------------------------------------------------------            
#County Trend Lines
            dcc.Graph(id='gen-line'),
            dcc.Dropdown(
                id='line-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P("(US Census Data)"),
            html.P('This graph displays the trends of uninsured rates in individual counties surrounding the clinic. Bedford, Warren, and Coffee county have the highest population of uninsured people, while most counties have an upward trend of uninsured percentages between 2020 and 2021. Marion County’s uninsured percentage is decreasing.'),
            html.Hr(),
            
            
#---------------------------------------------------------------------------------------------------------------------            
#Age
            dcc.Graph(id='age-graph'),
            dcc.Slider(
                id="year-slider",
                min=emp_census['year'].min(),
                max=emp_census['year'].max(),
                step=1,
                value=emp_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2023)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='age-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P('(US Census Data)'),
            html.P('This graph displays uninsured percentages based on specific age ranges. On average, people within the age of 26 and 34 are more likely to be uninsured.'),
            html.Hr(),
            
#---------------------------------------------------------------------------------------------------------------------
#Employment
            dcc.Graph(id='emp-graph'),
            dcc.Slider(
                id="year-slider-emp",
                min=emp_census['year'].min(),
                max=emp_census['year'].max(),
                step=1,
                value=emp_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2023)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='emp-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P("(US Census Data)"),
            html.P('The employed and unemployed categories are part of the labor force while not in the labor force are people that are not actively searching for work. Coffee, Bedford, and Warren County have significantly more uninsured working people. They have about double the amount of the next closest county (Franklin County).'),
            html.Hr(),
#---------------------------------------------------------------------------------------------------------------------            
#Disability
            dcc.Graph(id='dis-graph'),
            dcc.Slider(
                id="year-slider-dis",
                min=dis_census['year'].min(),
                max=dis_census['year'].max(),
                step=1,
                value=dis_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2022)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='dis-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P("(US Census Data)"),
            html.P('Being on disability one of the exceptions for the clinics eligibility. Information on which counties with higher disability populations will be taken into account when targeting new patients. People on disability are slightly less likely to be uninsured except for Moore, Bedford, and Warren which have significant differences. '),
            html.Hr(),
#---------------------------------------------------------------------------------------------------------------------            
#Poverty Section
            dcc.Graph(id='pov-graph'),
            dcc.Slider(
                id="year-slider-pov",
                min=pov_census['year'].min(),
                max=pov_census['year'].max(),
                step=1,
                value=pov_census['year'].max(),
                marks={year: str(year) for year in range(2018, 2022)},
                className="slider"
            ),
            html.P(''),
            dcc.Dropdown(
                id='pov-dropdown',
                options=[
                    {'label': 'Uninsured Population', 'value': 'uninsured'},
                    {'label': 'Percent Uninsured', 'value': 'percent_uninsured'},
                ],
                value='percent_uninsured'
            ),
            
            html.P(''),
            html.P('(US Census Data)'),
            html.P('Provides insight on areas PfH should target new patients based on Federal poverty level.Moore County showed significant improvement in their poverty level, while Cannon and Warren County are noticeably declining. Partners for Healing may want to concentrate less on Moore and focus more attention on Warren and Cannon'),
            html.Hr(),
            ]
    
#---------------------------------------------------------------------------------------------------------------------
# ER PATHNAME
#---------------------------------------------------------------------------------------------------------------------
    elif pathname == "/ER":
        return [
            html.H1("Preventable ER Visits", style={'textAlign': 'center'}),
            
            html.P("Every year Emergency Departments are overused by ___%. There are many factors that contribute to this issue. Some patients misuse the ER and go in for conditions/issues that could have been resolved without the use of the Emergency Department. Another issue is that many patients are going in for preventable conditions. These preventable visits are classified as Ambulatory Care Sensitive (ASC) Conditions. ACS conditions are conditions that would have been prevented if the patient had consistent primary care. The lack of primary care often leads to manageable/treatable conditions escalating  to the point where they need the ER. Using the 2019 TN Health Data we analyzed how often these preventable visits are occuring in the Franklin, Coffee, and Moore area."),
            html.Hr(),
            
            html.P("Text explainging ER Usage and preventable visits"),
            html.P("Table of preventable visits with corresponding IDC-10 codes"),
            html.P("Graph of # of preventable visits by County"),
            html.P("Graph of # of preventable visits within the ZIPCODE"),
            
#             dcc.Graph(id='bargraph',
#                       figure=create_line_graph(gen_census, counties_to_show))
        ]
    
#---------------------------------------------------------------------------------------------------------------------
# RURAL VS URBAN PATHNAME
#---------------------------------------------------------------------------------------------------------------------
    elif pathname == "/rural_urban":
        return [
            html.H1("Rural vs Urban Healthcare", style={'textAlign': 'center'}),
            
            html.P("Tab Description"),
            html.Hr(),
            
            html.P("Graph of top ten ICD-10 by population percentage and color as rural or urban"),
            html.P("Graph description"),
            html.Hr(),
            html.P("Graph of (rural) top ten ICD-10 codes by population percentage with colors as self pay or insured"),
            html.P("Graph of (Urban) top ten ICD-10 codes by population percentage with colors as self pay or insured"),

#             dcc.Graph(id='bargraph',
#                       figure = create_line_graph(gen_census, counties_to_show) #added yearly uninsured graph)
                        ]
# Define callbacks to update the content based on the selected sub-tabs

#---------------------------------------------------------------------------------------------------------------------
# RUN THE APPLICATION
#---------------------------------------------------------------------------------------------------------------------
if __name__=='__main__':
    app.run_server(debug=True, port=8010)
#http://127.0.0.1:8002/county